# Getting Started with Orchestrator

This notebook demonstrates the basic features of the Orchestrator framework with interactive examples.

## Installation

First, make sure you have Orchestrator installed:

```bash
pip install orchestrator-ai
```

Or clone and install from source:
```bash
git clone https://github.com/ContextLab/orchestrator
cd orchestrator
pip install -e .
```

## Setup

Let's start by importing the necessary modules and setting up our API keys.

In [ ]:
import os

# Import Orchestrator
from orchestrator import Orchestrator, init_models

# Set API keys (replace with your actual keys)
# os.environ['OPENAI_API_KEY'] = 'your-openai-key'
# os.environ['ANTHROPIC_API_KEY'] = 'your-anthropic-key'

# Check if API keys are set
has_openai = bool(os.getenv('OPENAI_API_KEY'))
has_anthropic = bool(os.getenv('ANTHROPIC_API_KEY'))

print(f"OpenAI API Key: {'✅ Set' if has_openai else '❌ Not set'}")
print(f"Anthropic API Key: {'✅ Set' if has_anthropic else '❌ Not set'}")

if not (has_openai or has_anthropic):
    print("\n⚠️ Warning: You'll need to set at least one API key to run AI-powered examples")

## Initialize Orchestrator

Let's create an Orchestrator instance with default models.

In [ ]:
# Initialize models and orchestrator
print("Initializing models...")
model_registry = init_models()
orchestrator = Orchestrator(model_registry=model_registry)

print(f"✅ Orchestrator initialized with {len(model_registry.models)} models")
print(f"Available models: {list(model_registry.models.keys())[:5]}...")

## Example 1: Simple Report Generation

Let's start with a simple pipeline that generates a report.

In [ ]:
simple_pipeline = """
name: simple-report
description: Generate a simple report

steps:
  - id: create_report
    tool: report-generator
    action: generate
    parameters:
      title: "My First Orchestrator Report"
      format: "markdown"
      content: |
        # Welcome to Orchestrator!
        
        This report was generated using the Orchestrator framework.
        
        ## Features:
        - YAML-based pipeline definitions
        - AI-powered task execution
        - Parallel processing
        - Intelligent model routing
        
        Generated at: {{ current_date }}
"""

# Execute the pipeline
result = await orchestrator.execute_yaml(simple_pipeline)
print("✅ Pipeline completed!")
print(f"Report content:\n{result['create_report']['content']}")

## Example 2: File Operations

Let's create a pipeline that works with files.

In [ ]:
file_pipeline = """
name: file-operations
description: Demonstrate file operations

steps:
  - id: create_file
    tool: filesystem
    action: write
    parameters:
      path: "/tmp/orchestrator_demo.txt"
      content: |
        This is a demo file created by Orchestrator.
        It contains sample data for processing.
        
        Data points:
        - Point 1: 42
        - Point 2: 84
        - Point 3: 126
      mode: "w"
      
  - id: read_file
    tool: filesystem
    action: read
    parameters:
      path: "/tmp/orchestrator_demo.txt"
      
  - id: check_exists
    tool: filesystem
    action: exists
    parameters:
      path: "/tmp/orchestrator_demo.txt"
"""

# Execute the pipeline
result = await orchestrator.execute_yaml(file_pipeline)
print("✅ File operations completed!")
print(f"File exists: {result['check_exists']['exists']}")
print(f"File content:\n{result['read_file']['content']}")

# Clean up
if os.path.exists("/tmp/orchestrator_demo.txt"):
    os.unlink("/tmp/orchestrator_demo.txt")
    print("✅ Cleanup completed")

## Example 3: AUTO Tags (AI-Powered)

This example demonstrates AUTO tags that use AI to make dynamic decisions.

In [ ]:
auto_tags_pipeline = """
name: auto-tags-demo
description: Demonstrate AUTO tags for dynamic decisions

inputs:
  user_query: "I need help creating a presentation about machine learning"

steps:
  - id: analyze_request
    tool: llm-generate
    action: generate
    parameters:
      prompt: "User request: {{ user_query }}"
      task_type: <AUTO>What type of task is this? Choose: 'presentation', 'analysis', 'coding', 'research'</AUTO>
      urgency: <AUTO>How urgent is this request? Choose: 'low', 'medium', 'high'</AUTO>
      
  - id: create_outline
    tool: llm-generate
    action: generate
    parameters:
      prompt: |
        Create an outline for: {{ user_query }}
        Task type: {{ analyze_request.task_type }}
        Urgency: {{ analyze_request.urgency }}
      detail_level: <AUTO>Given the urgency level, how detailed should this be? Choose: 'brief', 'standard', 'comprehensive'</AUTO>
      
  - id: generate_report
    tool: report-generator
    action: generate
    parameters:
      title: "AI-Generated Task Response"
      format: <AUTO>What's the best format for a {{ analyze_request.task_type }} deliverable? Choose: 'markdown', 'html', 'pdf'</AUTO>
      content: |
        # Task Analysis
        
        **Original Request:** {{ user_query }}
        
        **Task Type:** {{ analyze_request.task_type }}
        **Urgency:** {{ analyze_request.urgency }}
        **Detail Level:** {{ create_outline.detail_level }}
        
        ## Generated Outline
        {{ create_outline.result }}
        
        ---
        *Generated by Orchestrator AUTO tags*
"""

# Execute only if we have API access
if has_openai or has_anthropic:
    result = await orchestrator.execute_yaml(auto_tags_pipeline, {
        "user_query": "I need help creating a presentation about machine learning"
    })
    print("✅ AUTO tags pipeline completed!")
    print(f"Task type identified: {result['analyze_request']['task_type']}")
    print(f"Urgency level: {result['analyze_request']['urgency']}")
    print(f"Detail level chosen: {result['create_outline']['detail_level']}")
    print(f"Output format: {result['generate_report']['format']}")
else:
    print("⚠️ Skipping AUTO tags example - requires API key")

## Example 4: Data Processing

Let's process some sample data using the data processing tools.

In [ ]:
# Create sample CSV data
sample_csv = """name,age,city,salary
Alice,28,New York,75000
Bob,35,San Francisco,95000
Carol,42,Chicago,65000
David,29,Seattle,80000
Eve,31,Boston,70000"""

# Write sample data to file
with open("/tmp/sample_data.csv", "w") as f:
    f.write(sample_csv)

data_pipeline = """
name: data-processing-demo
description: Process CSV data

steps:
  - id: read_csv
    tool: filesystem
    action: read
    parameters:
      path: "/tmp/sample_data.csv"
      
  - id: transform_data
    tool: data-processing
    action: transform
    parameters:
      input_data: "{{ read_csv.content }}"
      input_format: "csv"
      output_format: "json"
      operations:
        - type: filter
          condition: "age > 30"
        - type: sort
          by: "salary"
          ascending: false
          
  - id: create_summary
    tool: report-generator
    action: generate
    parameters:
      title: "Data Processing Summary"
      format: "markdown"
      content: |
        # Data Processing Results
        
        ## Filtered Data (Age > 30, sorted by salary)
        
        {% for person in transform_data.result %}
        - **{{ person.name }}**: Age {{ person.age }}, {{ person.city }}, ${{ person.salary }}
        {% endfor %}
        
        Total records processed: {{ transform_data.result | length }}
"""

# Execute the pipeline
result = await orchestrator.execute_yaml(data_pipeline)
print("✅ Data processing completed!")
print(f"Processed {len(result['transform_data']['result'])} records")
print("\nSummary report:")
print(result['create_summary']['content'])

# Clean up
if os.path.exists("/tmp/sample_data.csv"):
    os.unlink("/tmp/sample_data.csv")

## Example 5: Parallel Execution

This example shows how Orchestrator can run independent tasks in parallel.

In [ ]:
import time

parallel_pipeline = """
name: parallel-demo
description: Demonstrate parallel task execution

steps:
  # These three tasks will run in parallel (no dependencies)
  - id: task_a
    tool: terminal
    action: execute
    parameters:
      command: "echo 'Task A completed' && sleep 2"
      timeout: 10
      
  - id: task_b
    tool: terminal
    action: execute
    parameters:
      command: "echo 'Task B completed' && sleep 1"
      timeout: 10
      
  - id: task_c
    tool: terminal
    action: execute
    parameters:
      command: "echo 'Task C completed' && sleep 3"
      timeout: 10
  
  # This task depends on all previous tasks
  - id: combine_results
    tool: report-generator
    action: generate
    dependencies: ["task_a", "task_b", "task_c"]
    parameters:
      title: "Parallel Execution Results"
      format: "markdown"
      content: |
        # Parallel Task Results
        
        ## Task Outputs:
        - Task A: {{ task_a.stdout }}
        - Task B: {{ task_b.stdout }}
        - Task C: {{ task_c.stdout }}
        
        All tasks completed successfully in parallel!
"""

# Time the execution
start_time = time.time()
result = await orchestrator.execute_yaml(parallel_pipeline)
end_time = time.time()

print(f"✅ Parallel execution completed in {end_time - start_time:.1f} seconds")
print("(Tasks ran in parallel - total time should be close to the longest task)")
print("\nCombined results:")
print(result['combine_results']['content'])

## Example 6: Error Handling

Let's see how Orchestrator handles errors and continues execution.

In [ ]:
error_handling_pipeline = """
name: error-handling-demo
description: Demonstrate error handling

steps:
  - id: successful_task
    tool: terminal
    action: execute
    parameters:
      command: "echo 'This task succeeds'"
      
  - id: failing_task
    tool: filesystem
    action: read
    parameters:
      path: "/nonexistent/file.txt"  # This will fail
    on_failure: continue  # Continue pipeline even if this fails
    
  - id: recovery_task
    tool: filesystem
    action: write
    condition: "{{ failing_task.status == 'failed' }}"  # Only run if previous failed
    parameters:
      path: "/tmp/recovery.txt"
      content: "Recovery action executed because previous task failed"
      
  - id: final_report
    tool: report-generator
    action: generate
    dependencies: ["successful_task"]  # Only depends on successful task
    parameters:
      title: "Error Handling Report"
      format: "markdown"
      content: |
        # Error Handling Results
        
        - Successful task: ✅ {{ successful_task.stdout }}
        - Failing task: ❌ {{ failing_task.status }}
        - Recovery task: {% if recovery_task %}✅ Executed{% else %}⏭️ Skipped{% endif %}
        
        Pipeline continued despite the failing task!
"""

try:
    result = await orchestrator.execute_yaml(error_handling_pipeline)
    print("✅ Pipeline completed with error handling")
    print("\nFinal report:")
    print(result['final_report']['content'])
    
    # Check if recovery file was created
    if os.path.exists("/tmp/recovery.txt"):
        with open("/tmp/recovery.txt", "r") as f:
            print(f"\nRecovery file content: {f.read()}")
        os.unlink("/tmp/recovery.txt")
        
except Exception as e:
    print(f"Pipeline failed: {e}")

## Pipeline Status and Monitoring

Orchestrator provides ways to monitor pipeline execution and get metrics.

In [ ]:
# Get performance metrics
metrics = await orchestrator.get_performance_metrics()

print("📊 Orchestrator Performance Metrics:")
print(f"Total executions: {metrics['total_executions']}")
print(f"Successful executions: {metrics['successful_executions']}")
print(f"Failed executions: {metrics['failed_executions']}")
print(f"Average execution time: {metrics['average_execution_time']:.2f}s")
print(f"Running pipelines: {metrics['running_pipelines']}")

# Get execution history
history = orchestrator.get_execution_history(limit=5)
print(f"\n📜 Recent Executions ({len(history)} shown):")
for record in history:
    status_emoji = "✅" if record['status'] == 'completed' else "❌"
    print(f"{status_emoji} {record['pipeline_id']} - {record['status']} ({record.get('execution_time', 0):.1f}s)")

## Health Check

Let's check the health of all Orchestrator components.

In [ ]:
# Perform health check
health = await orchestrator.health_check()

print("🏥 System Health Check:")
for component, status in health.items():
    if status == 'healthy':
        emoji = "✅"
    elif status == 'warning':
        emoji = "⚠️"
    else:
        emoji = "❌"
    print(f"{emoji} {component}: {status}")

## Next Steps

Now that you've seen the basics of Orchestrator, here are some next steps to explore:

1. **Explore Examples**: Check out the `examples/` directory for more complex workflows
2. **Read the Documentation**: Visit the `docs/` directory for comprehensive guides
3. **Create Custom Tools**: Learn how to build your own tools for specific needs
4. **Advanced Features**: Explore model routing, AUTO tags, and parallel processing
5. **Integration**: Connect Orchestrator with your existing systems

### Useful Links

- [Documentation](../README.md)
- [Examples](../../examples/)
- [API Reference](../reference/api_reference.md)
- [Tool Catalog](../reference/tool_catalog.md)
- [GitHub Repository](https://github.com/ContextLab/orchestrator)

Happy orchestrating! 🎼

In [ ]:
# Clean up (optional)
print("🧹 Cleaning up...")
orchestrator.clear_execution_history()
print("✅ Cleanup completed")